# <u>Projet 5:Segmentez des clients d'un site e-commerce</u>

### <u>Sommaire</u>

- <a href ="#0"> Introduction</a>
- 1. <a href ="#1">Importer les librairies et les données</a>
- 2. <a href ="#2">Importer la fonction client()</a>
- 3. <a href ="#3">Le contrat de maintenance</a>
    - 3.1. <a href ="#31">Avec le modèle KMeans</a>
    - 3.2. <a href ="#32">Avec le modèle DBSCAN</a>
    - 3.2. <a href ="#33">Avec le modèle Agglomerative</a>

- <a href ="#c">Conclusion générale</a>


## <a name = "0">Introduction</a>

<div style="text-align: justify">
Le troisième NoteBook3 est pour déterminer la fréquence nécessaire de mise à jour du modèle de segmentation. Et c'est toujours avec la même fonction client().
</div>

## <a name = "1">1. Importer les librairies et les données</a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Les algorithmes
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split

#Radar Chart
import plotly.graph_objects as go

# Pour normaliser les valeurs avec MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

#Pour calculer ARI Score
from sklearn.metrics.cluster import adjusted_rand_score

<div style="text-align: justify">
Pour comprendre la relation entre les données, regardez le schéma:<a href= https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce > Schéma Customers Dataset </a>
</div>

In [2]:
Cus=pd.read_csv('./DataProjet5/olist_customers_dataset.csv') #Customers
Geo=pd.read_csv('./DataProjet5/olist_geolocation_dataset.csv') #Geolocation
OrdIt=pd.read_csv('./DataProjet5/olist_order_items_dataset.csv') #OrderItmes
OrdPa=pd.read_csv('./DataProjet5/olist_order_payments_dataset.csv') #OrderPayments
OrdRe=pd.read_csv('./DataProjet5/olist_order_reviews_dataset.csv') #OrderReviews
Ord=pd.read_csv('./DataProjet5/olist_orders_dataset.csv') #Orders
Pro=pd.read_csv('./DataProjet5/olist_products_dataset.csv') #Products
Sel=pd.read_csv('./DataProjet5/olist_sellers_dataset.csv') #Sellers
ProCatNamTran=pd.read_csv('./DataProjet5/product_category_name_translation.csv') #Product_Category_Name_Translation

## <a name = "2">2. Importer la fonction client()</a>

<div style="text-align: justify">
Le code complet pour ce projet est dans cette fonction. C'est plus simple et c'est pour ne pas répéter les mêmes codes pour les différentes demandes de ce projet. 
<br>
    
<b> <u>Références de Choice:</u> </b> 
<ul>
<li><b> Choice='Percentage'</b>_Le pourcentage des clients ayant achetés plus d'une commande</li>
<li><b> Choice='RFM'</b>_Un DataFrame contient les valeurs R, F, et M</li>
<li><b> Choice='Elblow'</b>_Un plot de Elbow pour choisir le optimal K (Meilleur nombre de clusters)</li>
<li><b> Choice='SilhouetteScore'</b>_Le score de Silhouette</li>
<li><b> Choice='RFMBox'</b>_Un BoxPlot par cluster</li>
<li><b> Choice='InfoCluster'</b>_Un DataFrame qui présente les informations des clients par cluster</li>
<li><b> Choice='RadarChart'</b>_Radar plot pour tous les clusers</li>
<li><b> Choice='Periode'</b>_Un DataFrame avec ARI score selon les périodes choisies</li>
</ul>       
    
<b> <u>Références des autres variables:</u> </b> 
<ul> 
<li><b>algo</b>_On a le choix entre trois algorithmes: 'KMeans','DBSCAN', ou 'Agglomerative'</li>
<li><b>t0</b>_La première période pour entrainer le modèle</li>
<li><b>nt</b>_La période ajoutée à chaque step pour ré_entrainer le modèle</li>
<li><b>n_clustersAgglomerative</b>_Le nombre de clusters pour l'algorithme Agglomerative</li>
<li><b>min_samplesDBSCAN</b>_Le paramètre min_samples pour l'algorithme DBSCAN</li>
<li><b>epsDBSCAN</b>_Le paramètre eps de l'algorithme DBSCAN (La distance maximum entre deux clusters)</li>
<li><b>OptimalK</b>_Le meilleur nombre de clusters pour l'algorithme KMeans</li>
<li><b>TestNewFeatures</b>_Pour ajouter et tester des nouvelles features</li>
<li><b>showfliers</b>_Pour afficher ou non les valeurs aberrantes dans BoxPlot</li>  
</ul>       
   
</div>

In [3]:
def client(Choice='RFM',algo='KMeans',t0=50,nt=100,n_clustersAgglomerative=5,min_samplesDBSCAN=2,epsDBSCAN=3,
           OptimalK=5,seed=6,TestNewFeatures=False,showfliers=False):
    
    # Pour avoir 'order_status', 'order_purchase_timestamp',
    #et ajouter 'payment_type', 'payment_installments', 'payment_value' 
    MergeAtOrders=Ord.merge(OrdPa, on='order_id') 
    MergeAtOrders=MergeAtOrders.drop_duplicates(subset=['order_id'], keep='first', ignore_index=True)
    # Pour ajouter 'customer_unique_id'
    MergeAtOrders=MergeAtOrders.merge(Cus, on='customer_id')
    MergeAtOrders=MergeAtOrders.drop_duplicates(subset=['customer_id'], keep='first', ignore_index=True)
    # Pour ajouter 'freight_value', 'price', et  'product_photos_qty'
    MergeProductsToOrderitems=OrdIt.merge(Pro, on='product_id')
    MergeAtOrders=MergeAtOrders.merge(MergeProductsToOrderitems, on='order_id') 
    MergeAtOrders=MergeAtOrders.drop_duplicates(subset=['order_id'], keep='first', ignore_index=True)
    # Pour ajouter 'review_score'
    MergeAtOrders=MergeAtOrders.merge(OrdRe, on='order_id') 
    MergeAtOrders=MergeAtOrders.drop_duplicates(subset=['order_id'], keep='first', ignore_index=True)
    # DataFrame final avec les commandes effectives
    df=MergeAtOrders.loc[MergeAtOrders['order_status']=='delivered'] 
    
    if Choice=='Percentage':
        ## Calculer le pourcentage des clients qui ont acheté plus d'une commande
        # Nombre de clients qui ont effectué plus d'une commande
        ClientsBoughtMoreThanOneOrder=len(df.loc[df['customer_unique_id'].duplicated()]['customer_unique_id'].unique()) 
        # Nombre de tous les clients
        TotalClients=len(df['customer_unique_id'].unique()) 
        # Pourcentage de clients qui ont effectué plus d'une commande
        ClientsBoughtMoreThanOneOrder_Percentage=100*ClientsBoughtMoreThanOneOrder/TotalClients 
        ClientsBoughtMoreThanOneOrder_Percentage=round(ClientsBoughtMoreThanOneOrder_Percentage,2)
        ClientsBoughtMoreThanOneOrder_Percentage=print("Pourcentage des clients ayant acheté plus d'une commande "
                                                        +str(ClientsBoughtMoreThanOneOrder_Percentage)+"%")
        return ClientsBoughtMoreThanOneOrder_Percentage
    
    ## Avoir les valeur de RFM
    # Regrouper les valeurs par client
    RFM_=df.groupby(['customer_unique_id']).agg({ x:list for x in ['order_purchase_timestamp','payment_value']})
    # Créé un DataFrame avec les valuers de RFM
    RFM=pd.DataFrame()
    RFM['R']=RFM_['order_purchase_timestamp'].apply(lambda x: pd.to_datetime(x).max()) 
    RFM['RFloat']=RFM['R'].apply(lambda x: float(x.strftime("%Y%m%d%H%M%S.%f")))
    RFM['F']=RFM_['payment_value'].apply(lambda x: len(x))
    RFM['M']=RFM_['payment_value'].apply(lambda x: sum(x))
    RFM=RFM.sort_values(by='R')
    if Choice=='RFM': return RFM
        
    if TestNewFeatures==False:
        X=RFM.iloc[:,1:]  
        dfClusters=RFM.copy()
    if TestNewFeatures==True:
        InfosForClusters=df.groupby(['customer_unique_id']).agg({
        'payment_type':'unique','review_score':'mean',
        'payment_installments':'mean','freight_value':'mean', 'price':'mean', 'product_photos_qty':'mean'
        })
        dfClusters=pd.concat([RFM,InfosForClusters],axis=1)
        dfClusters=dfClusters.dropna(subset=['product_photos_qty'])
        X=dfClusters.drop(columns=['payment_type','R'])
        
    if Choice=='Elblow':
        if algo=='KMeans':
            Inertia=[]
            RandomColor='#%06X' % np.random.randint(0, 0xFFFFFF)
            for number_of_clusters in range(2,15): 
                kmeans = KMeans(n_clusters = number_of_clusters, random_state = seed)
                kmeans.fit(X) 
                # Méthode du coude 'Eblow'
                Inertia.append(kmeans.inertia_) 
            Elblow=pd.DataFrame()
            Elblow['K(number_of_clusters)']=range(2,15)
            Elblow['Inertia']=Inertia
            Elblow=Elblow.sort_values(by='Inertia')        
            x=Elblow['K(number_of_clusters)']
            y=Elblow['Inertia']
            fig,ax=plt.subplots(figsize=(10,6))
            ax.set_facecolor('white')
            plt.title('Eblow method for optimal k', size=20)
            plt.xlabel('K(number_of_clusters)',fontsize=14,fontweight='bold')
            plt.ylabel('Inertia',fontsize=14,fontweight='bold')
            fig=plt.plot(x,y,color=RandomColor,linewidth=3)
            fig=plt.plot(x,y,'or',markersize=7)      
            return fig 

    if algo=='KMeans':
        kmeans = KMeans(n_clusters=OptimalK, random_state=seed)
        kmeans.fit(X)
        Labels=kmeans.labels_
        algo=kmeans
    if algo=='DBSCAN' or algo=='Agglomerative':
        dfClusters_train, dfClusters_test= train_test_split(dfClusters, test_size=0.65, random_state=seed)
        dfClusters=dfClusters_train
        if TestNewFeatures==False:
            X_train=dfClusters.iloc[:,1:]  
        if TestNewFeatures==True:
            X_train=dfClusters.drop(columns=['payment_type','R'])
        X=X_train
    if algo=='DBSCAN':
        clustering = DBSCAN(eps=epsDBSCAN, min_samples=min_samplesDBSCAN)
        clustering .fit(X)
        Labels=clustering.labels_
        algo=clustering
    if algo=='Agglomerative':
        clustering = AgglomerativeClustering(n_clusters=n_clustersAgglomerative)
        clustering.fit(X)
        Labels=clustering.labels_
        algo=clustering

    dfClusters['Labels']=Labels
    dfClusters['NumberOfClient']=1
    
    if Choice=='SilhouetteScore':
        # Coefficient de silhouette
        Silhouette=silhouette_score(X, Labels, metric='euclidean', random_state=seed) 
        Silhouette=round(100*Silhouette,2)
        SilhouetteScore=print('SilhouetteScore est:'+str(Silhouette)+' %') 
        return SilhouetteScore
     
    if Choice=='RFMBox': 
        dfRFM=dfClusters
        SortedLabels=dfRFM['Labels'].sort_values().unique()
        dfRFM=dfRFM.set_index(['Labels'])
        dfR=pd.DataFrame()
        dfF=pd.DataFrame()
        dfM=pd.DataFrame()
        for label in SortedLabels: 
            dfOneCluster=dfRFM.loc[label]
            dfR[label]=dfOneCluster['RFloat'].reset_index(drop=True)
            dfF[label]=dfOneCluster['F'].reset_index(drop=True)
            dfM[label]=dfOneCluster['M'].reset_index(drop=True)
        dfForSns=[dfR,dfF,dfM]
        yLabels=['R','F','M']
        i=0
        allfig=[]
        for df in dfForSns:
            fig,ax=plt.subplots(figsize=(6,6))
            ax.set_facecolor('white')
            plt.title('Cluster Segmentation', size=18)
            plt.xlabel('Label of cluster',fontsize=13,fontweight='bold')
            plt.ylabel(yLabels[i],fontsize=13,fontweight='bold')
            i+=1
            sns.boxplot(data=df,width=0.6,fliersize=2,showfliers=showfliers)
            allfig.append(fig)
        return allfig
    
    if Choice=='InfoCluster' or Choice=='RadarChart':
        if TestNewFeatures==False:
            dfClusters=dfClusters.groupby(['Labels']).agg({'NumberOfClient':'sum','R':list, 'F':'mean', 'M':'mean'})
        if TestNewFeatures==True:  
            dfClusters=dfClusters.groupby(['Labels']).agg({
                'NumberOfClient':'sum',
                'R':list, 'F':'mean', 'M':'mean',
                'review_score':'mean','payment_type':list,
                'payment_installments':'mean','freight_value':'mean', 'price':'mean', 'product_photos_qty':'mean'})
            ToFillPaymentType=[]
            for label in dfClusters.index:
                PaymentTypeList=[]
                for ListPaymentType in dfClusters['payment_type'][label]:
                    for PaymentType in ListPaymentType:
                        PaymentTypeList.append(PaymentType)
                ToFillPaymentType.append(pd.Series(PaymentTypeList).unique())
            dfClusters['payment_type']=ToFillPaymentType
       
        ToFillMeanDeltaDate=[]
        for label in dfClusters.index:
            Period=pd.Series(dfClusters['R'][label])
            Period=Period.max()-Period.min()
            Period=Period/dfClusters['NumberOfClient'][label]
            ToFillMeanDeltaDate.append(Period)
        dfClusters['MeanDeltaDate']=ToFillMeanDeltaDate
        dfClusters=dfClusters.drop(columns=['R']) 
       
        if Choice=='InfoCluster':
            dfClusters=dfClusters.round(decimals = 2)
            return dfClusters
        if Choice=='RadarChart' :
            scaler = MinMaxScaler(feature_range=(0,1))
            if TestNewFeatures==False:
                dfRadar=dfClusters
            if TestNewFeatures==True:    
                dfRadar=dfClusters.drop(columns=['payment_type'])
            dfRadar['MeanDeltaDate']=dfRadar['MeanDeltaDate'].apply(lambda x: x.seconds)
            dfRadarScaled_=scaler.fit_transform(dfRadar)
            dfRadarScaled=pd.DataFrame(dfRadarScaled_,columns=dfRadar.columns)
            fig = go.Figure()
            for label in dfRadarScaled.index:
                fig.add_trace(go.Scatterpolar(r=dfRadarScaled.loc[label], theta=dfRadarScaled.columns,fill='toself',name='Cluster'+str(label)))
            fig.update_layout(polar=dict(radialaxis=dict(visible=True, range=[0, 1])), showlegend=True)
            return fig 

    if Choice=='Periode':
        TimedeltaZero=pd.to_datetime(0)-pd.to_datetime(0)
        DeltaDate=[TimedeltaZero]
        for i in range(len(RFM)-1):
            x1=RFM['R'][i]
            x2=RFM['R'][i+1]    
            DeltaDate.append(x2-x1)
        RFM['DeltaDate']=DeltaDate
        RFM['DeltaDate']=RFM['DeltaDate'].cumsum()
        RFM['DeltaDate']=RFM['DeltaDate'].apply(lambda x: x.days)
        dfClusters['DeltaDate']=RFM['DeltaDate']
        LastDate=RFM['DeltaDate'][len(RFM)-1]
        t=t0
        ARIScore=[]
        ARIScoreIndex=[]
        while t < LastDate:
            range_t=range(0,t)
            dfClustersRanget=dfClusters.loc[dfClusters['DeltaDate'].isin(range_t)]
            labels_true=dfClustersRanget['Labels']
            if TestNewFeatures==False:
                X=dfClustersRanget[['RFloat','F','M']]
            if TestNewFeatures==True:
                X=dfClustersRanget.drop(columns=['payment_type','R'])
            labels_pred=algo.fit(X).labels_         
            ARIScore.append(adjusted_rand_score(labels_true,labels_pred))
            ARIScoreIndex.append(t)
            t=t+nt
        ARIScore=pd.DataFrame(ARIScore,columns=['ARIScore']).round(4)
        ARIScore.index=ARIScoreIndex
        ARIScore['>0.8']= ARIScore['ARIScore']>0.8
        return ARIScore

## <a name = "3">3. Le contrat de maintenance</a>

### <a name = "31">3.1. Avec le modèle KMeans</a>

<div style="text-align: justify">
Avec Choice='Periode' on obtient les scores ARI entre les vrais labels et les lables prévues pour chaque période. Les vrais lables sont celles calculées pour toute la période. Les lables prévues sont celles calculées pour une partie de la période totale. Range(0,t0) est la première période pour la quelle le modèle est entrainé. Range(0,t0+nt) est la prochaine période pour laquelle le modèle est ré-entrainé. Toutes les périodes commence par 0 jusqu'à la période donnée par index au DataFrame ARIScore.
</div>

In [5]:
ARIScore=client(Choice='Periode',algo='KMeans',t0=50,nt=100)

In [6]:
ARIScore

,ARIScore,>0.8
50,0.0000,False
150,0.1374,False
250,0.0316,False
350,0.3737,False
450,0.4261,False
550,0.6993,False
650,0.8252,True


<div style="text-align: justify">
ARIScore est moins de 0.8 jusqu'à la période 550. A partir de 650 le score est plus de 0.8. On considère le score 0.8 comme un bon indicateur pour le contrat de maintenance. C'est à dire au dessous de cette valeur le modèle est obsolète et nécessite d'être ré-entrainé. On va essayer des valeurs à partir de t0=600.
</div>

In [7]:
ARIScore=client(Choice='Periode',algo='KMeans',t0=600,nt=15)

In [8]:
ARIScore

,ARIScore,>0.8
600,0.6437,False
615,0.8376,True
630,0.8304,True
645,0.8261,True
660,0.8228,True
675,1.0000,True
690,1.0000,True


<div style="text-align: justify">
A partir de la période 615 le score est toujours plus de 0.8.
</div>

In [9]:
ARIScore=client(Choice='Periode',algo='KMeans',t0=50,nt=100,TestNewFeatures=True)

In [10]:
ARIScore

,ARIScore,>0.8
50,0.0000,False
150,0.1365,False
250,0.0321,False
350,0.3751,False
450,0.4255,False
550,0.6982,False
650,0.8240,True


<div style="text-align: justify">
Ajouter des nouvelles features ne fait pas des grands changements pour le score ARI.
</div>

### <a name = "32">3.2. Avec le modèle DBSCAN</a>

<div style="text-align: justify">
Le modèle DBSCAN n'est pas un bon modèle à utiliser pour le projet mais on peut quand même analyser son contrat de maintenance.
</div>

In [11]:
ARIScore=client(Choice='Periode',algo='DBSCAN',t0=50,nt=100)

In [12]:
ARIScore

,ARIScore,>0.8
50,1.0,True
150,1.0,True
250,1.0,True
350,1.0,True
450,1.0,True
550,1.0,True
650,1.0,True


<div style="text-align: justify">
ARIScore est toujours plus de 0.8. C'est à dire que ce modèle n'est jamais obsolète. On va essayer de changer la période. 
</div>

In [13]:
ARIScore=client(Choice='Periode',algo='DBSCAN',t0=10,nt=40)

In [14]:
ARIScore

,ARIScore,>0.8
10,1.0,True
50,1.0,True
90,1.0,True
130,1.0,True
170,1.0,True
210,1.0,True
250,1.0,True
290,1.0,True
330,1.0,True
370,1.0,True


<div style="text-align: justify">
ARIScore est toujours plus de 0.8. 
</div>

In [15]:
ARIScore=client(Choice='Periode',algo='DBSCAN',t0=10,nt=40,min_samplesDBSCAN=2)

In [16]:
ARIScore

,ARIScore,>0.8
10,1.0,True
50,1.0,True
90,1.0,True
130,1.0,True
170,1.0,True
210,1.0,True
250,1.0,True
290,1.0,True
330,1.0,True
370,1.0,True


<div style="text-align: justify">
Le changement de min_samplesDBSCAN n'a pas fait la différence.
</div>

In [17]:
ARIScore=client(Choice='Periode',algo='DBSCAN',t0=10,nt=40,TestNewFeatures=True)

In [18]:
ARIScore

,ARIScore,>0.8
10,1.0,True
50,1.0,True
90,1.0,True
130,1.0,True
170,1.0,True
210,1.0,True
250,1.0,True
290,1.0,True
330,1.0,True
370,1.0,True


<div style="text-align: justify">
Ajouter des nouvelles features ne fait pas la différence.
</div>

### <a name = "33">3.3. Avec le modèle Agglomerative</a>

<div style="text-align: justify">
Comme Agglomerative prend plus du temps que KMeans, l'obtention des résultats va prendre plus de temps.
</div>

In [19]:
ARIScore=client(Choice='Periode',algo='Agglomerative',t0=50,nt=100)

In [20]:
ARIScore

,ARIScore,>0.8
50,0.0000,False
150,0.1302,False
250,0.0299,False
350,-0.0166,False
450,0.5378,False
550,0.8857,True
650,0.6954,False


<div style="text-align: justify">
Pour Agglomerative, ce n'est pas vraiment évident à quelle période on peut séparer entre obsolète ou non. Car à la période 550 le modèle n'est pas obsolète mais ensuite il revient obsolète. Cela confirme que Agglomerative est sensible à l'ordre de données. On va ré-essayer à partir de la période 500. 
</div>

In [23]:
ARIScore=client(Choice='Periode',algo='Agglomerative',t0=500,nt=15)

In [24]:
ARIScore

,ARIScore,>0.8
500,0.8362,True
515,0.8534,True
530,0.8680,True
545,0.8811,True
560,0.8931,True
575,0.9040,True
590,0.9159,True
605,0.9218,True
620,0.8521,True
635,0.7427,False


<div style="text-align: justify">
On remarque le même problème qu'on ne peut pas vraiment déterminer à quelle période le modèle est obsolète.
</div>

## <a name = "c">Conclusion générale</a>

<div style="text-align: justify">
Encore un projet intéressant dans cette formation. Dans chaque projet, il y a des nouvelles idées à appliquer et à apprendre. Par exemple, le schéma de données était pour comprendre la relation entre les données et pour combiner les DataFrames avec merge. Aussi l'utilisation de dates plusieurs fois, cela aide à être à l'aise avec les formats de date. J'ai également appris comment déterminer le contrat de maintenance pour un modèle selon la période.<br>   
Ce que j'aime faire pour tous les projets est de donner toutes les solutions possibles. Dans ce projet, j'ai pu mettre toute l'analyse dans une seule fonction. Avec cette fonction on peut avoir toutes les possibilités et toutes les solutions qu'on souhaite. A la fin, je suis toujours motivé pour la suite.

</div>